<a href="https://colab.research.google.com/github/Declan-Bracken/Khthon/blob/master/scrape_news_website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install requests beautifulsoup4

In [15]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [24]:
pip install pyngrok

In [17]:
import streamlit as st
import requests
from bs4 import BeautifulSoup
import re
import json


In [25]:
%%writefile app.py
import streamlit as st
import requests
from bs4 import BeautifulSoup
import re
import json

def extract_articles_from_script(script_content):
    pattern = re.compile(r'da2a.targets=\[(.*?)\];', re.DOTALL)
    match = pattern.search(script_content)
    if not match:
        st.write("No articles found in the script.")
        return []

    json_str = '[' + match.group(1).replace('url:', '"url":').replace('title:', '"title":') + ']'
    try:
        articles = json.loads(json_str)
    except json.JSONDecodeError as e:
        st.write(f"Error decoding JSON: {e}")
        return []

    return articles

def get_article_content(url):
    response = requests.get(url)

    if response.status_code != 200:
        st.write(f"Failed to retrieve content from {url}. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    article_body = soup.find('div', class_='article-body')  # Example class; change as needed
    if article_body:
        return article_body.get_text(strip=True)
    else:
        return soup.get_text(strip=True)  # Fallback to raw text if no specific class is found

def search_articles(keyword):
    url = search_url + keyword
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        st.write(f"Failed to retrieve data for {keyword}. Status code: {response.status_code}")
        return

    script_content = response.text
    articles = extract_articles_from_script(script_content)

    if not articles:
        st.write(f"No articles found for {keyword}.")
        return

    for article in articles:
        title = article['title']
        article_url = article['url']
        st.subheader(title)
        st.write(f"[Read Article]({article_url})")

        if st.button(f"Read More - {title}"):
            content = get_article_content(article_url)
            if content:
                st.text_area(f"Content - {title}", content, height=300)
            else:
                st.write("Content could not be retrieved.")

# Streamlit app layout
st.title("Article Search and Display for Boko Haram Related Topics")

# Keywords related to Boko Haram
keywords = ["Boko Haram", "Sambisa Forest", "Borno State", "Chibok", "Maiduguri"]

# Correct search URL for the African Union website
search_url = "https://au.int/en/search/node/"

# Define headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}

# Iterate over the keywords and perform the search
for keyword in keywords:
    st.write(f"### Searching for articles with keyword: {keyword}")
    search_articles(keyword)
    st.write("-" * 80)


Writing app.py


In [37]:
from pyngrok import ngrok

# Set your authtoken
ngrok.set_auth_token("2lHkyp8C4dfxUUNd7DnfFh882pM_7w2rCXXt98NpHnUwUHuoQ")

# Start the Streamlit app in the background
!streamlit run app.py --server.port 8001

# Wait for a few seconds to allow Streamlit to start
import time
time.sleep(5)

# Then connect ngrok, explicitly specifying HTTP protocol
public_url = ngrok.connect(8000, proto='http') # Use integer for port, and specify protocol
print(f"Public URL: {public_url}")





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8001
  Network URL: http://172.28.0.12:8001
  External URL: http://34.85.148.43:8001

  Stopping...
  Stopping...
Public URL: NgrokTunnel: "https://d676-34-85-148-43.ngrok-free.app" -> "http://localhost:8000"


In [38]:
# Keywords related to Boko Haram
keywords = ["Boko Haram", "Sambisa Forest", "Borno State", "Chibok", "Maiduguri"]

# Correct search URL for the African Union website
search_url = "https://au.int/en/search/node/"

# Define headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}


2024-08-28 10:56:43.908 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-08-28 10:56:44.072 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-08-28 10:56:44.074 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [4]:
"""  def extract_articles_from_script(script_content):
      # Print part of the script content to help with debugging
      if "advagg_mod_defer_1()" in script_content:
          index = script_content.index("advagg_mod_defer_1()")
          print("Script snippet around function call:")
          print(script_content[index-100:])  # Adjust the range as needed

      # Use a regular expression to find the JSON-like part in the script
      pattern = re.compile(r'da2a.targets=\[(.*?)\];', re.DOTALL)
      match = pattern.search(script_content)
      if not match:
          print("No articles found in the script.")
          return []

      # Extract the JSON-like string
      json_str = '[' + match.group(1).replace('url:', '"url":').replace('title:', '"title":') + ']'

      # Debugging: Print the JSON string
      print("Extracted JSON string:")
      print(json_str[:1000])  # Print a portion of the JSON string to verify its format


      # Parse the JSON string to a Python list
      try:
          articles = json.loads(json_str)
      except json.JSONDecodeError as e:
          print(f"Error decoding JSON: {e}")
          return []

      return articles
      """

In [43]:
def extract_articles_from_script(script_content):
    # Locate the position of the function call "advagg_mod_defer_1()"
    if "advagg_mod_defer_1()" in script_content:
        index = script_content.index("advagg_mod_defer_1()")
        # Extract the script content starting from 100 characters before the function call
        snippet = script_content[index-100:]
        print("Script snippet around function call:")
        #print(snippet[:500])  # Adjust the range as needed for debugging

        # Use a regular expression to find the JSON-like part in the script snippet
        pattern = re.compile(r'da2a.targets=\[(.*?)\];', re.DOTALL)
        match = pattern.search(snippet)
        if not match:
            st.write("No articles found in the script.")
            return []

        # Extract the JSON-like string
        json_str = '[' + match.group(1).replace('url:', '"url":').replace('title:', '"title":') + ']'

        # Debugging: Print the JSON string
        print("Extracted JSON string:")
        print(json_str[:1000])  # Print a portion of the JSON string to verify its format

        # Parse the JSON string to a Python list
        try:
            articles = json.loads(json_str)
        except json.JSONDecodeError as e:
            st.write(f"Error decoding JSON: {e}")
            return []

        return articles
    else:
        print("Function 'advagg_mod_defer_1()' not found in the script content.")
        return []


In [40]:
def get_article_content(url):
    # Make a request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve content from {url}. Status code: {response.status_code}")
        return None

    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the article content (you may need to adjust the tag/class based on the actual HTML structure)
    article_body = soup.find('div', class_='article-body')  # Example class; change as needed
    if article_body:
        return article_body.get_text(strip=True)
    else:
        return soup.get_text(strip=True)  # Fallback to raw text if no specific class is found

In [ ]:
"""def search_articles(keyword):
    # Format the search URL with the keyword
    url = search_url + keyword

    # Make an HTTP request to the website with headers
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve data for {keyword}. Status code: {response.status_code}")
        return

    # Extract the relevant part of the script content
    script_content = response.text

    # Extract articles from the script content
    articles = extract_articles_from_script(script_content)

    if not articles:
        print(f"No articles found for {keyword}.")
        return

    for article in articles:
        title = article['title']
        url = article['url']
        print(f"Keyword: {keyword}\nTitle: {title}\nLink: {url}\n")
        """

In [41]:
def search_articles(keyword):
    url = search_url + keyword
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to retrieve data for {keyword}. Status code: {response.status_code}")
        return

    script_content = response.text
    articles = extract_articles_from_script(script_content)

    if not articles:
        print(f"No articles found for {keyword}.")
        return

    for article in articles:
        title = article['title']
        article_url = article['url']
        print(f"Title: {title}\nURL: {article_url}\n")

        # Get the content of the article
        content = get_article_content(article_url)
        if content:
            print(f"Content: {content[:500]}...")  # Display first 500 characters of the content
        else:
            print("Content could not be retrieved.\n")


In [44]:
# Search for articles for each keyword
for keyword in keywords:
    print(f"Searching for articles with keyword: {keyword}")
    search_articles(keyword)
    print("-" * 80)

Searching for articles with keyword: Boko Haram
Failed to retrieve data for Boko Haram. Status code: 404
--------------------------------------------------------------------------------
Searching for articles with keyword: Sambisa Forest
Failed to retrieve data for Sambisa Forest. Status code: 404
--------------------------------------------------------------------------------
Searching for articles with keyword: Borno State
Failed to retrieve data for Borno State. Status code: 404
--------------------------------------------------------------------------------
Searching for articles with keyword: Chibok
Script snippet around function call:
Extracted JSON string:
[
{"title":"AU welcomes the release of 21 Chibok girls ","url":"https:\/\/au.int\/en\/pressreleases\/20161016"},
{"title":"AU welcomes the release of 21 Chibok girls ","url":"https:\/\/au.int\/en\/pressreleases\/20161016"},
{"title":"Interview with Special Envoy for Women Peace \u0026 Security","url":"https:\/\/au.int\/en\/art